In [ ]:
!nvidia-smi

Wed Jun 23 02:47:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#mounting drive to access the files in it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 27.5MB/s 
     |████████████████████████████████| 3.3MB 48.0MB/s 
     |████████████████████████████████| 901kB 48.9MB/s 


In [ ]:
#importing required libraries
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import numpy as np
import pandas as pd


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels,label_dict):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [ ]:
def evaluate(dataloader_val,model,device):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:
      
      batch = tuple(b.to(device) for b in batch)
      
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
              }

      with torch.no_grad():        
          outputs = model(**inputs)
          
      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

  loss_val_avg = loss_val_total/len(dataloader_val) 

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
          
  return loss_val_avg, predictions, true_vals

In [ ]:
def dec(result):
    if(result==0):
        return("PERSON")

    elif(result==1):
        return("LOCATION")

    elif(result==2):
        return("NUMBER")

    elif(result==3):
        return("MONEY")

    elif(result==4):
        return("DATE")

    elif(result==5):
        return("TIME")
    
    elif(result==6):
        return("PERCENTAGE")

    else:
        return("ORGANISATION")

In [ ]:
def BERT(input_query):
  
  qa=open("/content/drive/MyDrive/data1.txt","a",encoding="utf-8")
  qa.write("\n")
  qa.write("    :"+input_query)
  qa.close()

  data = open('/content/drive/MyDrive/data1.txt',encoding="utf-8").read()
  labels, texts = [], []

  for i, line in enumerate(data.split("\n")):
      content = line.split(":")
      labels.append(content[0])
      texts.append(" ".join(content[1:]))

  df = pd.DataFrame()
  df['text'] = texts
  df['tag'] = labels
  df.head()

  df['tag'].value_counts()

  possible_labels = df.tag.unique()

  label_dict = {}
  for index, possible_label in enumerate(possible_labels):
      label_dict[possible_label] = index
  label_dict

  df['label'] = df.tag.replace(label_dict)
  df.head()

  from sklearn.model_selection import train_test_split

  # X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
  #                                                   df.label.values, 
  #                                                   test_size=0.15, 
  #                                                   random_state=42, 
  #                                                   stratify=df.label.values)

  X_train = df.index.values[:1073]
  y_train = df.label.values[:1073] 

  X_val = df.index.values[1073:len(df.index.values)]
  y_val = df.label.values[1073:len(df.label.values)] 
  X_train.shape,X_val.shape

  df['data_type'] = ['not_set']*df.shape[0]

  df.loc[X_train, 'data_type'] = 'train'
  df.loc[X_val, 'data_type'] = 'val'
  df.groupby(['tag', 'label', 'data_type']).count()
  
  tokenizer = BertTokenizer.from_pretrained('kuppuluri/telugu_bertu_tydiqa', 
                                          do_lower_case=True)

  encoded_data_train = tokenizer.batch_encode_plus(
      df[df.data_type=='train'].text.values, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=150, 
      return_tensors='pt'
  )

  encoded_data_val = tokenizer.batch_encode_plus(
      df[df.data_type=='val'].text.values, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=150, 
      return_tensors='pt'
  )


  input_ids_train = encoded_data_train['input_ids']
  attention_masks_train = encoded_data_train['attention_mask']
  labels_train = torch.tensor(df[df.data_type=='train'].label.values)
  #print(input_ids_train.[0])
  #print(attention_masks_train[0])

  input_ids_val = encoded_data_val['input_ids']
  attention_masks_val = encoded_data_val['attention_mask']
  labels_val = torch.tensor(df[df.data_type=='val'].label.values)
  #print(input_ids_val)
  #print(attention_masks_val)

  dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)  #combine the training inputs into a TensorDataset
  dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
  len(dataset_train), len(dataset_val)

  model = BertForSequenceClassification.from_pretrained("kuppuluri/telugu_bertu_tydiqa",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
  

  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  batch_size = 3

  dataloader_train = DataLoader(dataset_train, 
                                sampler=RandomSampler(dataset_train),  #selects batches randomly
                                batch_size=batch_size)


  dataloader_validation = DataLoader(dataset_val, 
                                    sampler=SequentialSampler(dataset_val),   #pull out batches sequentially 
                                    batch_size=batch_size)
  

  from transformers import AdamW, get_linear_schedule_with_warmup

  optimizer = AdamW(model.parameters(),
                    lr=1e-5, 
                    eps=1e-8)
  
  epochs = 10

  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0,
                                              num_training_steps=len(dataloader_train)*epochs)

  
  import random

  seed_val = 17
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  print(device)

  for epoch in tqdm(range(1, epochs+1)):

    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

      model.zero_grad()
      
      batch = tuple(b.to(device) for b in batch)
      
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
              }       

      outputs = model(**inputs)
      
      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()
      
      progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation,model,device)
    val_f1 = f1_score_func(predictions[:len(true_vals)], true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  model = BertForSequenceClassification.from_pretrained("kuppuluri/telugu_bertu_tydiqa",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

  model.to(device)

  model.load_state_dict(torch.load('finetuned_BERT_epoch_10.model', map_location=torch.device('cpu')))

  _, predictions, true_vals = evaluate(dataloader_validation,model,device)

  accuracy_per_class(predictions[:len(true_vals)], true_vals, label_dict)

  return predictions




In [ ]:
input_query = "రాజ్యాంగంలో పేర్కొన్న మొత్తం ప్రాథమిక విధుల సంఖ్య ఎంత?"

predictions = BERT(input_query)

print(predictions)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Some weights of the model checkpoint at kuppuluri/telugu_bertu_tydiqa were not used when initializin

cuda



Epoch 1
Training loss: 0.7669656748945861
hii
hii2
Validation loss: 0.7717696678323241
F1 Score (Weighted): 0.7808659887311864



Epoch 2
Training loss: 0.33275909620849675
hii
hii2
Validation loss: 0.8744008864281229
F1 Score (Weighted): 0.8013793522692892



Epoch 3
Training loss: 0.20143989559364447
hii
hii2
Validation loss: 0.970650058658026
F1 Score (Weighted): 0.7975835058716271


...


Epoch 8
Training loss: 0.00407578217705605
hii
hii2
Validation loss: 0.9129006123582187
F1 Score (Weighted): 0.8545317032270019



Epoch 9
Training loss: 0.0012223579144130029
hii
hii2
Validation loss: 0.950970058553279
F1 Score (Weighted): 0.8546070947853339



Epoch 10
Training loss: 0.0010407161930616289
hii
hii2
Validation loss: 0.9523418478250049
F1 Score (Weighted): 0.8546070947853339



Some weights of the model checkpoint at kuppuluri/telugu_bertu_tydiqa were not used when initializing BertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kuppuluri/telugu_bertu_tydiqa and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

Class: PERS
Accuracy: 72/80

Class: LOCA
Accuracy: 71/71

Class: NUMB
Accuracy: 40/43

Class: MONE
Accuracy: 5/7

Class: DATE
Accuracy: 35/36

Class: TIME
Accuracy: 4/11

Class: PERC
Accuracy: 10/11

Class: ORGA
Accuracy: 1/7

Class:     
Accuracy: 0/6

[[-1.141906   -1.7148986  -1.9256299  ... -0.44608358 -0.9219713
  -0.6516167 ]
 [-1.038403   -1.2284648   9.23155    ... -1.083798   -1.538359
  -0.9109939 ]
 [-1.1889795  -1.2710899   9.292618   ... -0.6432615  -1.5105842
  -0.9542438 ]
 ...
 [-0.98046416 -1.6062834   9.223303   ... -0.89775836 -1.3429582
  -1.0278356 ]
 [-0.98046416 -1.6062835   9.223303   ... -0.8977582  -1.3429581
  -1.0278354 ]
 [-0.98046416 -1.6062835   9.223303   ... -0.8977582  -1.3429581
  -1.0278354 ]]


In [ ]:
res = predictions[-1].argmax(axis=-1)

res = dec(res);
print(res)

NUMBER


cuda



Epoch 1
Training loss: 0.6892188395165201
Validation loss: 0.5097131968003267
F1 Score (Weighted): 0.8545370820595054



Epoch 2
Training loss: 0.2986513979055943
Validation loss: 0.5168419257188633
F1 Score (Weighted): 0.8881909382218035



Epoch 3
Training loss: 0.16222283797657217
Validation loss: 0.509096492482384
F1 Score (Weighted): 0.9042992263334807



Epoch 4
Training loss: 0.10033227107610161
Validation loss: 0.5531015147753894
F1 Score (Weighted): 0.8963850669565517



Epoch 5
Training loss: 0.05241403508805793
Validation loss: 0.5961105892840433
F1 Score (Weighted): 0.901249086532107



Epoch 6
Training loss: 0.030638508220267556
Validation loss: 0.5587834255173003
F1 Score (Weighted): 0.9031923714759535



Epoch 7
Training loss: 0.021974753000770425
Validation loss: 0.6053009178483365
F1 Score (Weighted): 0.8989203710766785



Epoch 8
Training loss: 0.010849039734488254
Validation loss: 0.5708997893568959
F1 Score (Weighted): 0.915785085561205



Epoch 9
Training loss: 0.006466129127549816
Validation loss: 0.5972254385278167
F1 Score (Weighted): 0.9152748163942194



Epoch 10
Training loss: 0.0016768729554697576
Validation loss: 0.5940762194220599
F1 Score (Weighted): 0.9152748163942194



<All keys matched successfully>